## Run with Union detectors
The template instrument can make the instrument with He3 tubes simulated with Union and NCrystal for the gas.

In [ ]:
import mcstasscript as ms
import template

In [ ]:
instrument = template.make(union_detectors=True,
                           input_path="instrument_code", output_path="data/data", NeXus=True)

## Insert sample
We insert a Union sample to go ith the Union simulation of detectors, we can reuse the same master by placing components before it.

In [ ]:
Sample_inc = instrument.add_component("Sample_inc", "Incoherent_process", before="master")
Sample_inc.sigma = 3.4176
Sample_inc.unit_cell_volume = 1079.1

Sample_pow = instrument.add_component("Sample_pow", "Powder_process", before="master")
Sample_pow.reflections = '"Na2Ca3Al2F14.laz"'

Sample = instrument.add_component("Sample", "Union_make_material", before="master")
Sample.process_string = '"Sample_inc,Sample_pow"'
Sample.my_absorption = 100*2.9464/1079.1

sample = instrument.add_component("sample", "Union_cylinder", before="master", RELATIVE="sample_position")
sample.set_parameters(radius=0.03, yheight=0.03, material_string='"Sample"', priority=5)

## Run the instrument

In [ ]:
data = instrument.backengine()

In [ ]:
print(data)

In [ ]:
ms.make_sub_plot(data, log=True)

## Read with McStasToX
There is still more work to be done to easily get data of this type to scipp, but McStasToX can still help.

In [ ]:
!pip install mcstastox

In [ ]:
import numpy as np
import mcstastox
import scipp as sc

with mcstastox.Read(data[0].original_data_location) as file:
    source_position, source_rotation = file.get_component_placement("Source")
    sample_position, sample_rotation = file.get_component_placement("sample_position")
    
    detector_comps = file.get_components_with_ids()
    global_positions_all = None    

    for detector in detector_comps:
        mon = ms.name_search(detector, data)

        y = mon.get_data_column("y")
        positions = np.column_stack((np.zeros_like(y), y, np.zeros_like(y)))
        global_positions = file.transform(positions, detector)

        if global_positions_all is None:
            global_positions_all = global_positions
            event_weights = mon.get_data_column("p")
            event_times = mon.get_data_column("t")
        else:
            global_positions_all = np.vstack((global_positions_all, global_positions))
            event_weights = np.concatenate((event_weights, mon.get_data_column("p")))
            event_times = np.concatenate((event_times, mon.get_data_column("t")))

    events = sc.DataArray(
            data=sc.array(dims=['events'], unit=sc.units.counts, values=event_weights),
            coords={
                'position' : sc.vectors(dims=['events'], values=global_positions_all, unit='m'),
                't': sc.array(dims=['events'], unit='s', values=event_times),
                'source_position': sc.vector(source_position, unit='m'),
                'sample_position': sc.vector(sample_position, unit='m'),                
            })

### Now we can analyze the data

In [ ]:
import plopp as pp

pp.scatter3d(events[0::3], pos='position', size=0.02, cbar=True, norm="linear")

In [ ]:
from scippneutron.conversion.graph.beamline import beamline
from scippneutron.conversion.graph.tof import elastic

# McStas provides absolute time, not time of flight
events.coords["tof"] = events.coords["t"]

graph = {**beamline(scatter=True), **elastic("tof")}

In [ ]:
events = events.transform_coords("dspacing", graph=graph)

In [ ]:
%matplotlib widget
events.hist(dspacing=500).plot(norm="linear")